In [1]:
import jupyter_black

jupyter_black.load()  # Python linter

# Automatically provide the LLM with the calculator tool

In [2]:
%cat ../baml_src/02_use_tool.baml

class ToolCall {
    @@dynamic
}

function UseTool(goal: string, output_format_prefix: string) -> ToolCall {
    client Default
    prompt #"
        Solve: {{ goal }}

        {{ ctx.output_format(prefix=output_format_prefix, or_splitter=" OR ")}}
    "#
}


In [3]:
from baml_agents import McpServers, view_prompt
from pydantic_ai.mcp import MCPServerStdio

from baml_client.type_builder import TypeBuilder
from baml_client.async_client import b


server_with_calculator_tool = MCPServerStdio(
    command="python",
    args=["-m", "mcp_server_calculator"],
)

async with McpServers([server_with_calculator_tool]) as server:
    # Add MCP tools to the BAML structured output schema
    tb, tool_runner, p = await server.build_tool_types(
        tb := TypeBuilder(),
        output_class=tb.ToolCall,
        tools=await server.list_tools(),
    )

    # View prompt
    goal = "Multiply all numbers between 10 and 15"
    request = await b.request.UseTool(goal, p, baml_options={"tb": tb})
    print(view_prompt(request))

    # LLM chooses the tools
    result = await b.UseTool(goal, p, baml_options={"tb": tb})

    # Run the tools
    tool_results = await tool_runner.run(result)

[system]
Solve: Multiply all numbers between 10 and 15

What are the next steps?

Answer in JSON format with one or multiple of the following intents

{
  intents: [
    {
      // Calculates/evaluates the given expression.
      intent: "calculate",
      expression: string,
    }
  ],
}


In [4]:
result

ToolCall(intents=[{'intent': 'calculate', 'expression': '11 * 12 * 13 * 14'}])

In [5]:
tool_results

[CallToolResult(meta=None, content=[TextContent(type='text', text='24024', annotations=None)], isError=False)]

## Prompt Customization example

In [6]:
from baml_agents import BamlToolPromptConfig


prompt_cfg = BamlToolPromptConfig(
    id_field="tool_id",
    tools_field="one_chosen_tool",
    can_select_many=False,  # Removes list relationship between tools
)

async with McpServers([server_with_calculator_tool]) as server:
    # Add MCP tools to the BAML structured output schema
    tb, tool_runner, p = await server.build_tool_types(
        tb := TypeBuilder(),
        output_class=tb.ToolCall,
        prompt_cfg=prompt_cfg,
    )

    # View prompt
    goal = "Multiply all numbers between 10 and 15"
    request = await b.request.UseTool(goal, p, baml_options={"tb": tb})
    print(view_prompt(request))

[system]
Solve: Multiply all numbers between 10 and 15

What are the next steps?

Answer in JSON format with one of the following tool_ids

{
  one_chosen_tool: {
    // Calculates/evaluates the given expression.
    tool_id: "calculate",
    expression: string,
  },
}
